In [64]:
import time
import os
import glob
import copy
import pathlib
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets, models, transforms

In [65]:
# Define run parmameters

transform_image_size = 224

batch_size = 8

num_epochs = 2

In [66]:
# Data dict

data_directory = "./dataset2/dorsal"

In [67]:
# Check device

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [68]:
# Increase number of unique images by factor of 2 by horizontal flipping,
# but cant vertically flip

# Change colour channels from 0-255 to 0-1, numpy to tensors

train_transformation = transforms.Compose([
    transforms.RandomResizedCrop(transform_image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406], # col = rgb channel
        [0.229, 0.224, 0.225]) # row = mean deviation
])

valid_transformation = transforms.Compose([
        transforms.Resize(transform_image_size),
        transforms.CenterCrop(transform_image_size),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225])
])

In [69]:
# Datasets

train_dataset = datasets.ImageFolder(os.path.join(data_directory, 'train'), train_transformation)
valid_dataset = datasets.ImageFolder(os.path.join(data_directory, 'valid'), valid_transformation)

# Dataloader

train_dataloader = DataLoader(
    train_dataset,
    batch_size = batch_size,
    shuffle = True, # Shuffle is True so the model is not biased to some categories
    num_workers=4 # How many sub processes to use for loading data into ram
)

valid_dataloader = DataLoader(
    valid_dataset,
    batch_size = batch_size,
    shuffle = True,
    num_workers=4
)

In [70]:
train_root = pathlib.Path(data_directory + '/train')
data_classes = sorted([item.name.split('/')[-1] for item in train_root.iterdir()])
num_classes = len(data_classes)

In [71]:
print(data_classes)

['female', 'male']


In [72]:
# CNN Network

input_channels = 512
output_channels = num_classes # set to the number of data classes, eg: 2 (male and female)
kernal_size = (1,1)
stride = (1,1)

conv_network = models.squeezenet1_0(True)
for parameter in conv_network.parameters():
            parameter.requires_grad = False
conv_network.classifier[1] = nn.Conv2d(input_channels, output_channels, kernel_size = kernal_size, stride = stride)
conv_network.num_classes = num_classes

In [73]:
model = conv_network.to(device)
print(model)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [74]:
params_to_update = model.parameters()
print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)
        
optimiser = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
loss_func = nn.CrossEntropyLoss()

Params to learn:
	 classifier.1.weight
	 classifier.1.bias


In [75]:
# Calc amount of training and validation images for each class

train_male = len(glob.glob(data_directory + '/train/male/*.jpg'))
train_female = len(glob.glob(data_directory + '/train/female/*.jpg'))
train_count = train_male + train_female

valid_male = len(glob.glob(data_directory + '/valid/male/*.jpg'))
valid_female = len(glob.glob(data_directory + '/valid/female/*.jpg'))
valid_count = valid_male + valid_female

print('train male:', train_male, '\ttrain female:', train_female, '\ttrain count:', train_count)
print('valid male:', valid_male, '\tvalid female:', valid_female, '\tvalid count:', valid_count)

train male: 1493 	train female: 3000 	train count: 4493
valid male: 463 	valid female: 724 	valid count: 1187


In [76]:
def run_phase(phase_type, phase_count, phase_dataloader, since):
    phase_accuracy = 0.0
    phase_loss = 0.0
    image_count = 0

    for images, labels in phase_dataloader:
        inputs = images.to(device)
        labels = labels.to(device)

        optimiser.zero_grad() # Zero out gradients when starting the training loop,
                              # else gradient will point away from objective direction

        with torch.set_grad_enabled(phase_type == 'train'):
            # Forward
            outputs = model(inputs)
            loss = loss_func(outputs, labels)

            _, prediction = torch.max(outputs, 1)

            if phase_type == 'train':
                # Backward
                loss.backward()
                optimiser.step()

        # Calc stats
        phase_accuracy = phase_accuracy + (torch.sum(prediction == labels.data))
        phase_loss = phase_loss + (loss.item() * images.size(0))

        # Update image counter and elpased time
        time_elapsed = time.time() - since
        time_min = time_elapsed // 60
        time_sec = time_elapsed % 60
        print('\r' + 'Image: ' + str(image_count), '\tElpased Time:', '{:.0f}m {:.0f}s'.format(time_min, time_sec), end='')
        image_count+=1

    epoch_accuracy = phase_accuracy.double() / phase_count
    epoch_loss = phase_loss / phase_count

    print('\n{} Loss: {:.4f} Acc: {:.4f}'.format(phase_type, epoch_loss, epoch_accuracy))
    
    return epoch_accuracy

In [77]:
# Model training
def train(model, train_dataloader, valid_dataloader, optimiser, num_epochs):
    best_acc = 0.0
    best_model = copy.deepcopy(model.state_dict())
    since = time.time()

    for epoch in range(num_epochs):

        print('Epoch', str(epoch + 1) + '/' + str(num_epochs))

        # Train model phase
        model.train()
        run_phase('train', train_count, train_dataloader, since)

        # Eval model phase
        model.eval()
        epoch_acc = run_phase('valid', valid_count, valid_dataloader, since)
        
        # Check performace
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model = copy.deepcopy(model.state_dict())

        print('\n')

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print(best_acc)
    
    return best_model

In [78]:
model = train(model, train_dataloader, valid_dataloader, optimiser, num_epochs)

Epoch 1/2
Image: 561 	Elpased Time: 3m 37s
train Loss: 0.3717 Acc: 0.8406
Image: 148 	Elpased Time: 4m 54s
valid Loss: 0.3470 Acc: 0.8425


Epoch 2/2
Image: 561 	Elpased Time: 8m 47s
train Loss: 0.2986 Acc: 0.8722
Image: 148 	Elpased Time: 9m 57s
valid Loss: 0.2879 Acc: 0.8888


Training complete in 10m 17s
tensor(0.8888, dtype=torch.float64)


In [79]:
print(model)

OrderedDict([('features.0.weight', tensor([[[[-7.8596e-03,  3.6351e-02,  5.9548e-02,  ..., -1.1042e-02,
           -3.1383e-02, -4.1490e-02],
          [-1.8588e-02,  4.4232e-02,  2.5275e-01,  ...,  1.1751e-01,
            7.1817e-02,  8.5236e-03],
          [-7.3618e-02, -1.4523e-01,  2.1080e-01,  ..., -5.9453e-02,
            7.0713e-02, -5.2098e-02],
          ...,
          [ 5.1393e-02, -1.1664e-01,  2.1728e-01,  ..., -5.8746e-02,
            2.0013e-01,  2.5179e-02],
          [-2.3340e-02, -1.1444e-01,  4.0975e-02,  ..., -1.8714e-01,
            3.3543e-02,  7.6814e-03],
          [ 5.9402e-04, -2.9829e-02,  9.1360e-02,  ..., -8.1534e-02,
            4.8496e-03, -5.4964e-03]],

         [[-4.6941e-03,  4.9783e-02,  6.0962e-02,  ...,  1.0213e-02,
            9.0631e-03, -1.8218e-02],
          [-3.8541e-02,  3.9837e-02,  2.4669e-01,  ...,  1.4264e-01,
            1.4346e-01,  4.6988e-02],
          [-1.0086e-01, -1.8138e-01,  1.8306e-01,  ..., -1.2363e-01,
            8.7764e-02,